## Načítanie dát

In [1]:
#načítanie potrebných knižníc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import os
import pandas as pd
import glob
#balíky pre strojové učenie
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [2]:
#os.getcwd()
#Extrahovanie názvov excelovských tabuliek v danom priečinku
folderpath = 'C:\\Users\\adamj\\Documents\\Projekty_Python\\data'
file_path = "C:\\Users\\adamj\\Documents\\Projekty_Python\\data"
file_name = list(os.listdir(file_path))
file_name = [file for file in file_name if file!='Financial_Indicators_Information.pdf']

In [3]:
#Načítanie všetkých excelovských tabuliek v danom priečinku do jedného listu
path = r'C:\\Users\\adamj\\Documents\\Projekty_Python\\data' # path of the folder where CSV files are stored
all_files = glob.glob(path + "/*.csv")
df_list= []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df_list.append(df)

data = df_list

## Predspracovanie dát

In [4]:
#Ukážka riedkosti záznamov pôvodnej tabuľky
#df_list[6].iloc[5111,:]

In [5]:
def weekly_to_monthly_data(df_list, x, column):
    dfmonth = df_list[x]
    dfmonth['DATE'] = pd.to_datetime(dfmonth['DATE'])
    dfmonth['Month_Number'] = dfmonth['DATE'].dt.month
    dfmonth['Year'] = dfmonth['DATE'].dt.year
    dfmonth['Day'] = '01'
    df2 = dfmonth.groupby(['Year','Month_Number']).agg({column:'mean','Month_Number':'max','Year':'max','Day':'max'}, as_index=False)
    df2['DATE'] = 0
    for i in range(len(df2)):
        df2.iloc[i,4] = str(df2.iloc[i,2]) + '-'+  str(df2.iloc[i,1])  + '-'+ str(df2.iloc[i,3])
    df2['DATE'] = pd.to_datetime(df2['DATE'])
    df2 = df2.drop(['Month_Number', 'Year', 'Day'], axis=1)
    return df2

In [6]:
#bank_credit: All Commercial Banks from z týždených na mesačné dáta
data[1] = weekly_to_monthly_data(df_list, 1, 'TOTBKCR')
#Continued Claims (Insured Unemployment)
data[4] = weekly_to_monthly_data(df_list, 4, 'CCSA')
#currency outside the U.S. Treasury, Federal Reserve Banks, and the vaults of depository institutions; (2) demand deposits at commercial banks 
data[9] = weekly_to_monthly_data(df_list, 9, 'WM1NS')
#savings deposits (including money market deposit accounts); (2) small-denomination time deposits 
data[10] = weekly_to_monthly_data(df_list, 10, 'WM2NS')

In [7]:
#before Column name is Capitalize
print(df_list[17].columns)
# converting all the column header to capital letter
for i in df_list:
    i.columns =  i.columns.str.upper()  
#After Column name is Capitalize
print(df_list[17].columns)

Index(['DATE', 'UNRATE'], dtype='object')
Index(['DATE', 'UNRATE'], dtype='object')


In [8]:
#vytvorenie nových dát ktoré začínajú od zadávaneho dátumu
for i in range(len(df_list)):
    df = df_list[i]
    res = df[~(df['DATE'] < '1983-01-01')]
    df_list[i] = res

In [9]:
#vymazanie nepotrebnej tabuľky a jej mena z listu mien tabuliek
del data[6]
del file_name[6]
#vymazanie nepotrebných stĺpcov v tabuľkách opisujúcih ceny indexov nasdaq and spx500
data[11]= data[11].drop(['HIGH','LOW'], axis=1)
data[12]= data[12].drop(['HIGH','LOW'], axis=1)

In [10]:
#interpolácia štvrťročných dát na mesačné 
def quarterly_to_monthly_data(df_list, x):
    temp = df_list[x]
    temp['DATE'] = pd.to_datetime(temp['DATE'])
    temp['DATE'] = pd.to_datetime(temp['DATE']).dt.to_period('M')
    temp = temp.set_index('DATE').resample('M').interpolate()
    temp =temp.reset_index()
    for i in range(len(temp)):
        temp.iloc[i,0] = df_list[1].iloc[i,1]
    return temp

In [11]:
#zmena atribútu v tabuľke 7 aby ju bolo možné spracovať funkciou
df_list[7].iloc[:,1] =    df_list[7].iloc[:,1].astype(float)

In [12]:
#intrapolácie mesačných dáť
#doplnenie posledného gdp
data[6] = quarterly_to_monthly_data(df_list,6)
data[7] = quarterly_to_monthly_data(df_list,7)

In [13]:
#manuálne pridanie záznamov
new_row = {'DATE':'2023-04-01', 'TOTBKCR':17333.9006}
data[1]=data[1].append(new_row, ignore_index=True)
new_row = {'DATE':'2023-04-01', 'CPIAUCSL':302.918}
data[2]=data[2].append(new_row, ignore_index=True)
new_rows = [{'DATE':'2023-02-01', 'CPALTT01USM657N':0.55821},
            {'DATE':'2023-03-01', 'CPALTT01USM657N':0.33107},
            {'DATE':'2023-04-01', 'CPALTT01USM657N':0.35123}]
data[3]=data[3].append(new_rows, ignore_index=True)
new_row = {'DATE':'2023-04-01', 'CCSA':1807000}
data[4]=data[4].append(new_row, ignore_index=True)
new_row = {'DATE':'2023-04-01', 'FEDFUNDS':4.83}
data[5]=data[5].append(new_row, ignore_index=True)
new_rows = [{'DATE':'2022-11-01', 'GDP':26137.9920},
            {'DATE':'2022-12-01', 'GDP':26187.9920},
            {'DATE':'2023-01-01', 'GDP':26237.9920},
            {'DATE':'2023-02-01', 'GDP':26337.9920},
            {'DATE':'2023-03-01', 'GDP':26387.9920},
            {'DATE':'2023-04-01', 'GDP':26486.287}]
data[6]=data[6].append(new_rows, ignore_index=True)
new_rows = [{'DATE':'2022-11-01', 'OEHRENWBSHNO':30681.345},
            {'DATE':'2022-12-01', 'OEHRENWBSHNO':30381.345},
            {'DATE':'2023-01-01', 'OEHRENWBSHNO':30181.345},
            {'DATE':'2023-02-01', 'OEHRENWBSHNO':29898.345},
            {'DATE':'2023-03-01', 'OEHRENWBSHNO':29598.345},
            {'DATE':'2023-04-01', 'OEHRENWBSHNO':29398.345}]
data[7]=data[7].append(new_rows, ignore_index=True)
new_row = {'DATE':'2023-04-01', 'WM1NS':18547.9}
data[8]=data[8].append(new_row, ignore_index=True)
new_row = {'DATE':'2023-04-01', 'WM2NS':20626.6}
data[9]=data[9].append(new_row, ignore_index=True)
new_row = {'DATE':'2023-04-01', 'MEDCPIM158SFRBCLE':4.91160}
data[10]=data[10].append(new_row, ignore_index=True)
new_row = {'DATE':'2023-04-01', 'CORESTICKM159SFRBATL':6.32275}
data[13]=data[13].append(new_row, ignore_index=True)
new_row = {'DATE':'2023-04-01', 'STICKCPIM157SFRBATL':0.38240}
data[14]=data[14].append(new_row, ignore_index=True)
new_row = {'DATE':'2023-04-01', 'UNEMPLOY':5739}
data[15]=data[15].append(new_row, ignore_index=True)
new_row = {'DATE':'2023-04-01', 'UNRATE':3.4}
data[16]=data[16].append(new_row, ignore_index=True)

In [65]:
#vypísanie počtu záznomov v jednotlivých tabuľkách  
for i in range(len(data)):
    print(len(data[i]))

484
484
484
484
484
484
484
484
484
484
484
10152
10152
484
484
484
484


In [66]:
for i in range(len(data)):
    data[i]["DATE"] = pd.to_datetime(data[i]["DATE"], errors='coerce')

In [67]:
file_name

['10-Year Real Interest Rate.csv',
 'Bank Credit All Commercial Banks.csv',
 'Consumer Price Index for All Urban Consumers All Items in U.S. City Average.csv',
 'Consumer Price Index Total All Items for the United States.csv',
 'Continued Claims (Insured Unemployment).csv',
 'Federal Funds Effective Rate.csv',
 'Gross Domestic Product.csv',
 'Households Owners Equity in Real Estate Level.csv',
 'M1.csv',
 'M2.csv',
 'Median Consumer Price Index.csv',
 'NASDAQ.csv',
 'SPX500.csv',
 'Sticky Price Consumer Price Index less Food and Energy.csv',
 'Sticky Price Consumer Price Index.csv',
 'Unemployment Level.csv',
 'Unemployment Rate.csv']

In [68]:
#creating metadata which will contain file_name, columns header, range of data
df_metadata = pd.DataFrame(file_name,columns=["File_Name"])

#collection of all column header of each and every file
df_header_list = []
for i in range(len(data)):
    df_header_list.append(list(data[i].columns))
    
min_date_list = []
max_date_list = []
for i in range(len(data)):
    min_date_list.append(data[i]["DATE"].min())
    max_date_list.append(data[i]["DATE"].max())
    
df_metadata["Columns"] = df_header_list
df_metadata["min_date"] = min_date_list
df_metadata["max_date"] = max_date_list
df_metadata

,File_Name,Columns,min_date,max_date
0,10-Year Real Interest Rate.csv,"[DATE, REAINTRATREARAT10Y]",1983-01-01,2023-04-01
1,Bank Credit All Commercial Banks.csv,"[TOTBKCR, DATE]",1983-01-01,2023-04-01
2,Consumer Price Index for All Urban Consumers A...,"[DATE, CPIAUCSL]",1983-01-01,2023-04-01
3,Consumer Price Index Total All Items for the U...,"[DATE, CPALTT01USM657N]",1983-01-01,2023-04-01
4,Continued Claims (Insured Unemployment).csv,"[CCSA, DATE]",1983-01-01,2023-04-01
5,Federal Funds Effective Rate.csv,"[DATE, FEDFUNDS]",1983-01-01,2023-04-01
6,Gross Domestic Product.csv,"[DATE, GDP]",1983-01-01,2023-04-01
7,Households Owners Equity in Real Estate Level.csv,"[DATE, OEHRENWBSHNO]",1983-01-01,2023-04-01
8,M1.csv,"[WM1NS, DATE]",1983-01-01,2023-04-01
9,M2.csv,"[WM2NS, DATE]",1983-01-01,2023-04-01


In [69]:
#spojenie 2 tabuliek
df_list[11].iloc[:,0] = pd.to_datetime(df_list[11].iloc[:,0])
merg = pd.merge(df_list[1],df_list[2], how='inner' )
merg = pd.merge(merg,df_list[3], how='inner' )
merg = pd.merge(merg,df_list[4], how='inner' )
merg = pd.merge(merg,df_list[5], how='inner' )
merg = pd.merge(merg,df_list[6], how='inner' )
merg = pd.merge(merg,df_list[7], how='inner' )
merg = pd.merge(merg,df_list[8], how='inner' )
merg = pd.merge(merg,df_list[9], how='inner' )
merg = pd.merge(merg,df_list[10], how='inner' )
merg = pd.merge(merg,df_list[11], how='left' )
merg = pd.merge(merg,df_list[13], how='inner' )
merg = pd.merge(merg,df_list[14], how='inner' )
merg = pd.merge(merg,df_list[15], how='inner' )
merg = pd.merge(merg,df_list[16], how='inner' )


In [70]:
merg['OPEN'] =  merg['OPEN'].interpolate(method='pad')
merg.iloc[0,11]= 248.00000
an_count = merg.isna().sum().sum()
an_count
#funkcia na získanie pozície s chýbajúcou hodnotou
#np.where(pd.isnull(merg))

0

In [71]:
merg

,TOTBKCR,DATE,CPIAUCSL,CPALTT01USM657N,CCSA,FEDFUNDS,GDP,OEHRENWBSHNO,WM1NS,WM2NS,MEDCPIM158SFRBCLE,OPEN,CORESTICKM159SFRBATL,STICKCPIM157SFRBATL,UNEMPLOY,UNRATE
0,1392.275100,1983-01-01,97.900,0.204918,3919000.0,8.68,3473.413,2474.362000,483.460,1960.760,5.014653,248.000000,4.695598,0.511700,11534,10.4
1,1399.634525,1983-02-01,98.000,0.102249,3859250.0,8.51,3508.558,2481.049000,474.100,1985.750,4.952545,248.149994,4.627836,0.354470,11545,10.4
2,1414.281060,1983-03-01,98.100,0.000000,3767500.0,8.77,3543.703,2487.736000,482.425,2008.650,3.612261,261.820007,4.855813,0.230150,11408,10.3
3,1426.981300,1983-04-01,98.800,0.715015,3760200.0,8.80,3578.848,2494.423000,499.175,2033.725,4.828054,261.820007,4.456934,0.521420,11268,10.2
4,1436.760000,1983-05-01,99.200,0.608519,3624250.0,8.63,3615.625,2498.634667,492.540,2030.140,3.767326,261.820007,3.549624,0.228230,11154,10.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,17514.441825,2022-12-01,298.990,-0.307009,1624800.0,4.10,26187.992,30381.345000,19912.450,21408.825,7.049161,11475.169922,6.617223,0.509330,5722,3.5
480,17552.152675,2023-01-01,300.536,0.799536,1660000.0,4.33,26237.992,30181.345000,19633.180,21269.240,8.142495,11475.169922,6.554492,0.506749,5694,3.4
481,17590.475350,2023-02-01,301.648,0.558210,1734000.0,4.57,26337.992,29898.345000,19345.800,21099.900,7.880496,11573.139648,6.593750,0.550846,5936,3.6
482,17497.030180,2023-03-01,301.808,0.331070,1804000.0,4.65,26387.992,29598.345000,19298.700,21123.300,4.907055,11447.580078,6.451708,0.383314,5839,3.5


In [72]:
merg["1monthpred"] = np.nan
merg["2monthpred"] = np.nan
merg["3monthpred"] = np.nan
merg["4monthpred"] = np.nan
merg["5monthpred"] = np.nan
merg["6monthpred"] = np.nan

## Modelovanie

In [201]:
#Pridanie závislej premennej na základe počtu mesiacov, ktorých dopredu chceme predvídať
for i in range(len(merg.iloc[:,1])-1):
    merg["1monthpred"].iloc[i] = merg["GDP"].iloc[i+1]
for i in range(len(merg.iloc[:,1])-2):
    merg["2monthpred"].iloc[i] = merg["GDP"].iloc[i+2]   
for i in range(len(merg.iloc[:,1])-3):
    merg["3monthpred"].iloc[i] = merg["GDP"].iloc[i+3]
for i in range(len(merg.iloc[:,1])-4):
    merg["4monthpred"].iloc[i] = merg["GDP"].iloc[i+4]
for i in range(len(merg.iloc[:,1])-5):
    merg["5monthpred"].iloc[i] = merg["GDP"].iloc[i+5]
for i in range(len(merg.iloc[:,1])-6):
    merg["6monthpred"].iloc[i] = merg["GDP"].iloc[i+6]

C:\Users\adamj\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [202]:
y = merg['1monthpred']
y= y.iloc[1:483]
x = merg[merg.columns.difference(['1monthpred','2monthpred','3monthpred','4monthpred','5monthpred','6monthpred','DATE'])]
x = x.iloc[1:483,:]

In [203]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=124421)
linear = LinearRegression()
linear1 = linear_model.BayesianRidge()
linear2 = linear_model.LassoLars(alpha=.1)

linear.fit(X_train, y_train)
linear1.fit(X_train, y_train)
linear2.fit(X_train, y_train)
#To retrieve the intercept:
#print(linear.intercept_)
#For retrieving the slope:
print(np.where(linear.coef_ == max(linear.coef_)))
print(np.where(linear1.coef_ == max(linear1.coef_)))
print(np.where(linear2.coef_ == max(linear2.coef_)))

(array([2], dtype=int64),)
(array([2], dtype=int64),)
(array([2], dtype=int64),)


In [204]:
X_train.columns[2]

'CPALTT01USM657N'

In [205]:
X_train

,CCSA,CORESTICKM159SFRBATL,CPALTT01USM657N,CPIAUCSL,FEDFUNDS,GDP,MEDCPIM158SFRBCLE,OEHRENWBSHNO,OPEN,STICKCPIM157SFRBATL,TOTBKCR,UNEMPLOY,UNRATE,WM1NS,WM2NS
416,1892800.0,2.106697,0.529491,246.435,1.15,19783.798667,2.870257,16721.699333,6442.169922,0.188040,12372.757275,6854,4.3,3535.600,13699.125
29,2551800.0,5.056542,0.279590,107.500,7.53,4356.144333,4.937369,3242.976667,280.500000,0.396510,1768.964775,8460,7.4,583.800,2408.675
132,2709600.0,3.384373,0.274348,146.300,3.05,7115.652000,2.711772,4946.046000,759.900024,0.203020,3007.192300,8630,6.6,1144.600,3481.800
295,2553000.0,2.468540,-0.183390,207.667,5.02,14614.430667,2.184573,12926.062667,2538.500000,0.205997,8219.319320,7067,4.6,1364.400,7344.950
82,2285000.0,4.811219,0.238854,125.900,8.55,5789.058333,4.162524,4726.758667,456.200012,0.382360,2530.360540,6725,5.4,790.625,3135.650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,4528600.0,0.749593,0.021104,217.605,0.18,15141.605000,0.632305,8933.253000,2110.750000,0.071298,8962.380925,14512,9.4,1712.725,8577.850
125,2812000.0,3.583429,0.138696,144.300,3.04,6857.711667,2.548574,4855.350000,700.960022,0.278820,2928.158780,9121,7.0,1071.575,3440.550
78,2179400.0,4.905248,0.241741,124.500,9.24,5695.365000,5.814260,4682.026000,446.799988,0.452530,2469.582475,6495,5.2,779.200,3051.940
140,2629250.0,3.267212,0.268456,149.300,4.73,7413.883667,3.747094,5025.434333,763.000000,0.218500,3129.036150,7734,5.9,1148.075,3479.375


In [206]:
y_test_predict = linear.predict(X_test)
y_train_predict= linear.predict(X_train)
y_test_predict1 = linear1.predict(X_test)
y_train_predict1= linear1.predict(X_train)
y_test_predict2 = linear2.predict(X_test)
y_train_predict2= linear2.predict(X_train)

In [207]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [208]:
len(y_test)

145

In [209]:
#Root-mean-square deviation metrika 
import numpy as np
print("RMSE",np.sqrt(mean_squared_error(y_test,y_test_predict)))
print("RMSE",np.sqrt(mean_squared_error(y_test,y_test_predict1)))
print("RMSE",np.sqrt(mean_squared_error(y_test,y_test_predict2)))

RMSE 81.78073510051975
RMSE 78.34662103405981
RMSE 50.49375150100307


In [210]:
#coefficient of determination metrika
from sklearn.metrics import r2_score
print( r2_score(y_test,y_test_predict))
print( r2_score(y_test,y_test_predict1))
print( r2_score(y_test,y_test_predict2))

0.9998199852015028
0.9998347860440386
0.9999313751409423


In [211]:
#predikcia na daľšie mesiace
x = merg[merg.columns.difference(['1monthpred','2monthpred','3monthpred','4monthpred','5monthpred','6monthpred','DATE'])]
last_month_data = x.iloc[483,:]
last_month_data = last_month_data.to_numpy().reshape(1,-1)
print(linear.predict(last_month_data))
print( linear1.predict(last_month_data))
print( linear2.predict(last_month_data))

[26673.76317942]
[26685.65578121]
[26561.13661903]


In [212]:
df = pd.DataFrame({'Actual': y_test, 'Predicted':y_test_predict})
df

,Actual,Predicted
478,26187.992000,26298.345271
385,18126.254000,18159.390679
191,9411.682000,9436.273981
73,5578.726333,5588.744687
39,4566.116333,4574.056621
...,...,...
342,15712.543000,15699.751510
111,6502.722333,6457.124924
127,6969.858000,6924.404934
287,14215.651000,14193.317107


In [213]:
#netóda na výpočet prázdnych hodnôt v tabuľke
#an_count = y.isna().sum().sum()
#an_count